In [1]:
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', None)  # me muestre todas las columnas
pd.set_option('display.max_colwidth', 100)  #me muestre más caracteres por columna.

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('attacks.csv', encoding= "ISO-8859-1") 

# Glosario de columnas y tipos iniciales

0. Case Number: La fecha coincide con el case number. OBJETC
1. Date: La fecha del ataque. OBJECT
2. Year: El año del ataque. FLOAT64
3. Type: Tipo de ataque (provocado, no provocado, desastre marino, etc). OBJECT
4. Country: País del ataque. OBJECT
5. Area: Area del país. OBJECT
6. Location: Localización concreta dentro del area. OBJECT
7. Activity: La actividad que se estaba realizando durante el ataque. OBJECT
8. Name: Nombre de la víctima. OBJECT
9. Sex: Género de la víctima. OBJECT
10. Age: Edad de la víctima. OBJECT
11. Injury: Tipo de lesión provocada. OBJECT
12. Fatal: Muerte o no de la victima. OBJECT
13. Time: Hora del ataque. OBJECT
14. Species: Especie del tiburón del ataque. OBJECT
15. Investigator or Source: Investigador del ataque + organización a la que pertenece. OBJECT
16. pdf: Imagino que un pdf del informe del ataque por víctima. OBJECT
17. href formula: Enlace al informe en pdf del apartado anterior. OBJECT
18. href: Parece una columna con lo mismo que lo anterior pero tiene menos filas. OBJECT
19. Case Number.1: Columna igual que Case Number pero con dos filas menos. OBJECT
20. Case Number.2: Columna igual que Case Number pero con una fila menos. OBJECT
21. Original Order: Parece un id del caso, a priori mayor cuanto más reciente. FLOAT64
22. Unnamed: 22: No sé qué significa, todo NaNs salvo el 1478 "stopped here". OBJECT
23. Unnamed: 23: No sé qué significa, todo NaNs salvo el 4415 "Teramo" y el 5840 "change filename". OBJECT

- Dimensión inicial del DataFrame: (25723, 24)

# Restricciones:
- No se pueden eliminar columnas.
- Deben quedar al menos 1500 filas.



# Primeras consideraciones

- Como no podemos eliminar columnas, el % de nulos por columna no es tan relevante.
- Sí podemos quitar todas las filas en las que todos los valores sean nulos. 
- Sí podemos quitar todas las filas duplicadas.
- Arreglar los nombres de las columnas para trabajar mejor con ellas.

In [3]:
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

In [4]:
data_ori_shape = data.shape

data_ori = data.copy()

data.shape

(25723, 24)

In [5]:
data = data.dropna(how='all')

In [6]:
data= data.drop_duplicates()

In [7]:
data.shape, data_ori.shape

((6311, 24), (25723, 24))

Tras hacerlo, las dimensiones pasan a ser: (6311, 24)

Ahora voy a cambiar los nombres de las columnas para homogeneizarlas, por si acaso.

In [8]:
data.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [9]:
data.columns = [e.replace(' ', '_') for e in data.columns]

data.columns = [e.replace('.', '_') for e in data.columns]

data.columns = [e.replace(':', '') for e in data.columns]

In [10]:
data.rename(columns={'Species_': 'Species', 'Sex_': 'Sex', 'Fatal_(Y/N)': 'Fatal', 'Investigator_or_Source': 'Source'}, inplace=True)

A continuación voy a eliminar las filas que tengan más de un 50% de valores nulos.

In [11]:
mask = data.isnull()                                      # Me devuelve una mascara booleana de toda la tabla.

In [12]:
null_values_count = mask.T.sum()                          # Me devuelve el nº de NaN que hay por fila.

In [13]:
nan_por_linea_100 = (null_values_count/len(data.T)) *100

In [14]:
mask2 = nan_por_linea_100 > 50 

In [15]:
'''La línea de código data = data[~mask2] es utilizada para filtrar las filas de un DataFrame en base 
a una máscara booleana.
La expresión ~mask2 invertirá los valores booleanos de la máscara, es decir, todos los valores True se 
convertirán en False y viceversa. Con esto, las filas que antes eran True (más del 50% de valores nulos) 
ahora serán False y serán eliminadas.
'''

data = data[~mask2]            

Las dimensiones del DataFrame pasan a ser: (6302, 24)

In [16]:
data.shape

(6302, 24)

En mi exploración me ha parecido que la columna original_order está ordenada de forma descendente. Procedo a comprobarlo y, de ser así, daré la columna por "buena". Tal vez más adelante, cuando haya terminado de eliminar filas, resetee esta columna para que vaya desde el 1 hasta el último caso y la renombre como una especie de número de caso, de id. De ser posible la convertiré también a integer.

In [17]:
print(data['original_order'].is_monotonic_decreasing)

True


In [18]:
data['original_order'] = data['original_order'].astype(int)

A continuación, centro mi atención en la columna 'Year', con el fin de tener ordenadas las filas al menos por año. Analizando el final del dataframe con .iloc, me doy cuenta de que los últimos años no tienen datos consistentes en la columna 'Year' a partir del índice 6173 en adelante. 

In [19]:
data['Year'].iloc[6172:]

6172    1554.0
6173    1543.0
6174     500.0
6175      77.0
6176       5.0
         ...  
6297       0.0
6298       0.0
6299       0.0
6300       0.0
6301       0.0
Name: Year, Length: 130, dtype: float64

Por tanto, es posible que decida eliminar las filas finales, pero primero quiero tener claro que el resto de años sí están ordenados.

In [20]:
data['Year'].iloc[:6173].is_monotonic_decreasing

False

No están ordenados, así que procedo a ordenar el dataframe según la columna. Después compruebo si ahora sí, salvo las últimas filas, está ordenados consistentemente.

In [21]:
data = data.sort_values(by='Year',ascending=False)
data = data.reset_index(drop=True)

Tras ordenarlo, parece que el nuevo indice a partir del cual empiezan los años consistentes es el 6171, así que a partir de él miro si están ordenados.

In [22]:
data['Year'].loc[:6171].is_monotonic_decreasing

True

En efecto, ya tengo la el dataframe ordenado por años, a falta de eliminar las últimas filas, lo cual procedo a realizar.

In [23]:
data = data.loc[:6171]

Voy a aprovechar para convertir los valores de la columna a enteros ya que los años son siempre enteros y así el Dataframe ocupará menos espacio.

In [24]:
data['Year'] = data['Year'].astype(int)

A continuación y tras analizar la columna 'Date', voy a intentar quedarme con el mes (ya tengo una columna con el año) en formato 1-12, con el fin de que ocupe menos y pueda manejar sus datos como númericos.

In [25]:
# Convierto la columna a formato fecha con pd.to_datetime.
# El parámetro errors='coerce' sirve para que los que no puedan convertirse pasen a ser NaN.
# Uso dt.month para asignar el valor númerico del mes a la propia columna 'Date'.

data['Date'] = pd.to_datetime(data['Date'], errors='coerce').dt.month

In [26]:
data['Date'].isna().value_counts()

False    5458
True      714
Name: Date, dtype: int64

Ahora voy a rellenar los NaN de la columna con el mes más repetido (moda), ya que la proporción de NaN me parece bastante baja en relación a los meses que sí tengo. Doy por hecho que si hay más ataques en enero en general, el valor menos adulterado será poner 1.0 (enero). Usar la media me daría meses con decimales, y rellenar con unknown o 0 me impediría trabajar correctamente con la columna.

In [27]:
data['Date'].fillna(data['Date'].mode()[0], inplace=True)

Una vez hecho esto, voy a cambiar el nombre de la columna por 'Month' y a convertir la columna a integers, ya que los meses son siempre enteros y así el Dataframe ocupará menos espacio. 

In [28]:
data.rename(columns={'Date': 'Month'}, inplace=True)

In [29]:
data['Month'] = data['Month'].astype(int)

Lo siguiente que voy a intentar es, ya que tengo mes y año, asignar a la columna 'Case_Number' el día del mes en que ocurrió el ataque. Tras analizar la columna, lo primero que voy a hacer es quitar los caracteres alfabéticos (letras) de la string, con el fin de dejar solo las fechas y extraer el día como he hecho con el mes.

In [30]:
data['Case_Number'] = data['Case_Number'].str.replace(r'[a-zA-Z]', '')

In [31]:
data['Case_Number'] = pd.to_datetime(data['Case_Number'], errors='coerce').dt.day

In [32]:
data['Case_Number'].isna().value_counts()

False    5417
True      755
Name: Case_Number, dtype: int64

En este caso, para los días del mes vacíos (NaN), voy a generar una serie de números aleatorios para rellenar dichos valores.

In [33]:
'''
Con el código siguiente, creo una lista vacía y lo que hago es iterar por todas las filas del DF. Si la columna 
'Case_Number' contiene un NaN, entonces añade a la lista un entero aleatorio entre 1 y 28 (días de mes) y, en
caso contrario, añade el valor actual a la lista. Finalmente relleno toda la columna con la lista obtenida.
entonces 
'''

import random

days = []

for i in data.index:
    
    if pd.isnull(data['Case_Number'].iloc[i]):
        
        days.append(random.randint(1,28))
    else:
        
        days.append(data['Case_Number'].iloc[i])
        
data['Case_Number'] = days

Ya tengo mi columna exactamente como la quiero, así que procedo a cambiarle el nombre y el tipo.

In [34]:
data.rename(columns={'Case_Number': 'Day'}, inplace=True)

In [35]:
data['Day'] = data['Day'].astype(int)

Ahora voy a por la columna 'Type'. Primero analizo la distribución de sus valores.

In [36]:
data.Type.value_counts()

Unprovoked      4483
Provoked         567
Invalid          544
Sea Disaster     233
Boating          203
Boat             135
Questionable       2
Boatomg            1
Name: Type, dtype: int64

Primero voy a unificar 'Boating', 'Boat' y 'Boatomg' como 'Boating'.

In [37]:
data['Type'] = data['Type'].replace({'Boatomg': 'Boat'})

In [38]:
data['Type'] = data['Type'].replace({'Boat': 'Boating'})

A continuación, voy a analizar los 'Questionable' con el fin de tomar una decisión sobre ellos.

In [39]:
data[data['Type'] == 'Questionable']

,Day,Month,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Time,Species,Source,pdf,href_formula,href,Case_Number_1,Case_Number_2,original_order,Unnamed_22,Unnamed_23
47,25,4,2018,Questionable,AUSTRALIA,New South Wales,Lennox Head,Surfing,Matthew Lee,M,NaN,No injury,N,07h00,Questionable,"B. Myatt, GSAF",2018.04.25.b-Lee.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.04.25.b-Lee.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.04.25.b-Lee.pdf,2018.04.25.b,2018.04.25.b,6283,NaN,NaN
51,9,5,2018,Questionable,AUSTRALIA,New South Wales,"Sharpes Beach, Ballina",Surfing,male,M,NaN,"No injury, surfboard damaged",N,10h30,Shark involvement not confirmed,"B. Myatt, GSAF",2018.05.09-SharpesBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.05.09-SharpesBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.05.09-SharpesBeach.pdf,2018.05.09,2018.05.09,6287,NaN,NaN


En la columna 'Species' se questiona el hecho de que un tiburón estuviera involucrado, y en ese caso no tendría sentido conservar estas líneas con sus datos. Además, la columna 'Injury' indica que no hubo lesiones, lo que refuerza la hipótesis. Por tanto, procedo a eliminar esas filas y ajustar el índice.

Nota: la columna 'original_order' perderá entonces su orden consecutivo, por lo que seguramente tendré que ajustarla después.

In [40]:
data = data.drop(data[data['Type'] == 'Questionable'].index)

In [41]:
data = data.reset_index(drop=True)

Ahora la columna 'Type' tiene solo cinco valores únicos, los cuales decido conservar. Procedo a remplazar los NaN por 'Invalid' y termino con ella.

In [42]:
data['Type'].fillna(value='Invalid', inplace=True)

A continuación, saco los valores únicos de la columna 'Country', con el fin de unificar o borrar en caso de necesidad.

In [43]:
data.Country.unique()

array(['USA', 'AUSTRALIA', 'BAHAMAS', 'BRAZIL', 'NEW CALEDONIA',
       'SOUTH AFRICA', 'ECUADOR', 'THAILAND', 'ENGLAND', 'MEXICO',
       'MALDIVES', 'COSTA RICA', 'UNITED ARAB EMIRATES',
       'ST HELENA, British overseas territory', 'REUNION', 'NEW ZEALAND',
       'UNITED KINGDOM', 'FRENCH POLYNESIA', 'SPAIN', 'COMOROS',
       'INDONESIA', 'PHILIPPINES', 'MAURITIUS', 'LIBYA', nan, 'CUBA',
       'SAMOA', 'MALAYSIA', 'EGYPT', 'SOLOMON ISLANDS', 'JAPAN',
       'COLUMBIA', 'CAPE VERDE', 'CAYMAN ISLANDS', 'DOMINICAN REPUBLIC',
       'Fiji', 'CHINA', 'PUERTO RICO', 'ATLANTIC OCEAN', 'ITALY',
       'MOZAMBIQUE', 'ARUBA', 'FIJI', 'FRANCE', 'ST. MARTIN',
       'TRINIDAD & TOBAGO', 'PAPUA NEW GUINEA', 'GREECE',
       'PALESTINIAN TERRITORIES', 'JAMAICA', 'TAIWAN', 'DIEGO GARCIA',
       'BELIZE', 'SEYCHELLES', 'GUAM', 'ISRAEL', 'KIRIBATI', 'CHILE',
       'SAUDI ARABIA', 'CROATIA', 'NIGERIA', 'TONGA', 'CANADA',
       'SCOTLAND', 'TURKS & CAICOS', 'UNITED ARAB EMIRATES (UAE)',
      

En el notebook en sucio 'draft' de esta misma carpeta, voy investigando caso por caso con el fin de adecuar 'Country' a lo que realmente corresponda, si es posible. Para ello utilizo la localización si existe, o incluso intento abrir el link con el pdf del suceso. En caso de no descubrirlo, tomaré una decisión para rellenarlo, lo mismo que con los NaN. 

In [44]:
data['Country'] = data['Country'].replace({'Fiji': 'FIJI', 'ST HELENA, British overseas territory': 'UNITED KINGDOM OVERSEAS', 'DIEGO GARCIA': 'UNITED KINGDOM OVERSEAS', 'Sierra Leone': 'SIERRA LEONE', 'Seychelles': 'SEYCHELLES',
                                 'EGYPT / ISRAEL': 'EGYPT', 'PACIFIC OCEAN ': 'PACIFIC OCEAN', 'BRITISH ISLES': 'UNITED KINGDOM', 'ENGLAND': 'UNITED KINGDOM', 'ST. MAARTIN': 'ST MARTIN', 'ST. MARTIN': 'ST MARTIN',
                                'NORTH ATLANTIC OCEAN ': 'NORTH ATLANTIC OCEAN', 'FEDERATED STATES OF MICRONESIA': 'MICRONESIA', 'BRITISH WEST INDIES': 'UNITED KINGDOM OVERSEAS', 'RED SEA / INDIAN OCEAN': 'RED SEA', 'ANDAMAN / NICOBAR ISLANDAS': 'BAY OF BENGAL',
                                 'SUDAN?': 'SUDAN', 'THE BALKANS': 'SLOVENIA', 'IRAN / IRAQ': 'IRAN', ' PHILIPPINES': 'PHILIPPINES', 'SOLOMON ISLANDS / VANUATU': 'VANUATU', 'ITALY / CROATIA': 'CROATIA', 'YEMEN ': 'YEMEN', 'REUNION': 'REUNION ISLAND',
                                'EGYPT ': 'EGYPT', 'BRITISH NEW GUINEA': 'UNITED KINGDOM OVERSEAS', 'OCEAN': 'PACIFIC OCEAN', 'INDIAN OCEAN?': 'INDIAN OCEAN', 'EQUATORIAL GUINEA / CAMEROON': 'CAMEROON', 'Coast of AFRICA': 'ATLANTIC OCEAN', 'Between PORTUGAL & INDIA': 'INDIAN OCEAN',
                                'TURKS & CAICOS': 'UNITED KINGDOM OVERSEAS', 'TRINIDAD & TOBAGO': 'TOBAGO', 'UNITED ARAB EMIRATES (UAE)': 'UNITED ARAB EMIRATES', 'BRITISH VIRGIN ISLANDS': 'UNITED KINGDOM OVERSEAS', ' TONGA': 'TONGA', 'MEXICO ': 'MEXICO', 'NICARAGUA ': 'NICARAGUA',
                                 'MID-PACIFC OCEAN': 'MID PACIFIC OCEAN'})

In [45]:
data['Country'].fillna(value='unknown', inplace=True)

In [46]:
data.Country.isna().unique()

array([False])

Así queda limpia otra columna. Sin embargo, en el análisis caso por caso he descubierto que el índice 6047, 'AFRICA' según la columna 'Country', no tiene prácticamente información, así que procedo a eliminar esta línea y reiniciar el índice.

In [47]:
data = data.drop(6047)

data = data.reset_index(drop=True)

Las columnas 'Area' y 'Location' contienen demasiados valores únicos como para revisarlos todos. De momento, voy a rellenar con 'unknown' todas aquellas filas que en ambas columnas tengan NaN. 

In [48]:
doble_nan = (data.Area.isna() == True) & (data.Location.isna() == True)

data.loc[doble_nan, ["Area","Location"]] = data.loc[doble_nan, ["Area","Location"]].fillna("unknown")

In [49]:
len(data[(data.Area.isna() == True) | (data.Location.isna() == True)])

495

Una vez reemplazados los NaN que coinciden en ambas filas, pretendía ver si con lo uno o con lo otro podía rellenar correctamente el Nan de al lado. Sin embargo, 495 filas parecen ser demasiadas como para ir una a una. No obstante, tras observar unas cuantas decenas de filas, me doy cuenta de que en muchos de los NaN que hay en 'Area' pueden ser porque no existe denominación intermedia entre el país y la localización, como en el caso de las islas. En el caso de 'Location', parece que en muchos casos simplemente no consta el lugar preciso. Como conclusión, y aunque seguro que podría rellenar alguna que otra fila, creo que lo más sensato es rellenar el NaN de una columna con lo que sí viene informado en la otra.

In [50]:
data.Area.fillna(data.Location, inplace=True)
data.Location.fillna(data.Area, inplace=True)

A continuación voy a investigar los principales valores de la columna Activity con el fin de unificar aquellos en los que proceda.

In [51]:
data.Activity.value_counts().head(40)

Surfing                 968
Swimming                853
Fishing                 423
Spearfishing            332
Bathing                 159
Wading                  147
Diving                  115
Standing                 97
Snorkeling               88
Scuba diving             75
Body boarding            61
Body surfing             49
Swimming                 47
Kayaking                 33
Fell overboard           32
Treading water           32
Boogie boarding          29
Pearl diving             28
Free diving              27
Windsurfing              19
Walking                  17
Boogie Boarding          16
Shark fishing            15
Floating                 14
Fishing                  13
Rowing                   12
Surf fishing             12
Surf-skiing              12
Surf skiing              12
Canoeing                 12
Fishing for sharks       11
Kayak Fishing            11
Scuba Diving             10
Freediving               10
Sailing                   9
Sitting on surfboard

In [52]:
# Genero una máscara booleana que devuelve True si 'urf' está contenido en el valor.

mask3 = data['Activity'].str.contains('urf')  

In [53]:
# Utilizo np.where con la mask como primer argumento, para que donde sea True complete con 'Surfing' (2º argumento).
# En caso de False, rellena con el valor correspondiente de data['Activity'] (3º argumento).

data['Activity'] = np.where(mask3, 'Surfing', data['Activity'])

Ahora voy a seguir el mismo procedimiento para intentar unificar el máximo de valores que sea posible.

In [54]:
data.Activity.value_counts().head(60)

Surfing                       1781
Swimming                       853
Fishing                        423
Spearfishing                   332
Bathing                        159
Wading                         147
Diving                         115
Standing                        97
Snorkeling                      88
Scuba diving                    75
Body boarding                   61
Swimming                        47
Kayaking                        33
Fell overboard                  32
Treading water                  32
Boogie boarding                 29
Pearl diving                    28
Free diving                     27
Walking                         17
Boogie Boarding                 16
Shark fishing                   15
Floating                        14
Fishing                         13
Canoeing                        12
Rowing                          12
Kayak Fishing                   11
Fishing for sharks              11
Freediving                      10
Scuba Diving        

In [55]:
data['Activity'] = np.where(data['Activity'].str.contains('urf'), 'Surfing', data['Activity'])

data['Activity'] = np.where(data['Activity'].str.contains('ishin'), 'Fishing', data['Activity'])

data['Activity'] = np.where(data['Activity'].str.contains('iving'), 'Diving', data['Activity'])

data['Activity'] = np.where(data['Activity'].str.contains('wim'), 'Swimming', data['Activity'])

data['Activity'] = np.where(data['Activity'].str.contains('ath'), 'Bathing', data['Activity'])

data['Activity'] = np.where(data['Activity'].str.contains('oard'), 'Boarding', data['Activity'])

data['Activity'] = np.where(data['Activity'].str.contains('ayak'), 'Boating', data['Activity'])

data['Activity'] = np.where(data['Activity'].str.contains('Boat'), 'Boating', data['Activity'])

data['Activity'] = np.where(data['Activity'].str.contains('boat'), 'Boating', data['Activity'])

data['Activity'] = np.where(data['Activity'].str.contains('Walking'), 'Wading', data['Activity'])

data['Activity'] = np.where(data['Activity'].str.contains('Standing'), 'Standing', data['Activity'])

data['Activity'] = np.where(data['Activity'].str.contains('loating'), 'Swimming', data['Activity'])

data['Activity'] = np.where(data['Activity'].str.contains('Canoe'), 'Boating', data['Activity'])

data['Activity'] = np.where(data['Activity'].str.contains('Sail'), 'Boating', data['Activity'])

In [56]:
data.Activity.value_counts().head(10)

Surfing       1781
Fishing       1118
Swimming      1114
Diving         482
Boarding       268
Bathing        195
Wading         167
Boating        150
Standing       119
Snorkeling      88
Name: Activity, dtype: int64

El resto de categorías, al ser tan poco representativas como confusas, voy a intentar unificarlas bajo el valor 'unknown'. Empezaré por los valores nulos.

In [57]:
data['Activity'].fillna(value='unknown', inplace=True)

In [58]:
# Primero creo un diccionario con los valores que quiero que se mantengan.

excepts = {'Surfing':'Surfing', 'Fishing':'Fishing', 'Swimming':'Swimming', 'Diving':'Diving', 'Boarding':'Boarding', 
                'Bathing':'Bathing', 'Wading':'Wading', 'Boating':'Boating', 'Standing':'Standing', 'Snorkeling':'Snorkeling'}

# Con data['Activity'].isin(excepts) uso otra máscara bool para seleccionar las filas cuyo valor está en excepts.
# Virgulilla ~ para negar esa condición, y que así el .loc elija las filas correctas de 'Activity' donde poner 'unknown'.

data.loc[~data['Activity'].isin(excepts), 'Activity'] = 'unknown'

A continuación analizo la columna 'Name'.

In [59]:
data.Name.value_counts().head(20)

male             520
female            95
boy               21
2 males           16
boat              14
child             12
sailor            10
Anonymous         10
a sailor           8
girl               6
Unidentified       6
males              6
a soldier          5
a native           5
fisherman          4
2 fishermen        4
Unknown            4
black male         4
a pearl diver      4
John Williams      3
Name: Name, dtype: int64

Aunque inicialmente se me ha ocurrido limpiar muchos falsos nombres, no estoy seguro de que deba tocar esta columna. En el cuaderno 'draft', primero unifiqué los valores con más de 4 ocurrencias (ya que no había ningún nombre real entre ellos) como 'No name'. Después hice lo mismo con todos aquellos valores que no tuvieran al menos una mayúscula. Finalmente intenté instalar spacy y NLTK con el fin de procesar el lenguaje y sacar los nombres, lo cual no conseguí. 

Entonces me percaté de que no iba a conseguir más nombres y que simplemente estaba quitando información de la columna sólo para sentirla ordenada, sustituyendo la mucha o poca información por un 'No name' que en principio no aporta mucho. Por ello he decidido mantenerla como está, al menos de momento, con la salvedad de rellenar los nulos como 'unknown'.

In [60]:
data['Name'].fillna(value='unknown', inplace=True)

En cuanto a la columna 'Sex', la exploración incial devuelve que, además de los nulos, hay unos pocos valores mal informados ('M ', 'lli', '.' o 'N'). Voy a cambiarlos para que reflejen lo que corresponda en el caso de poder, y en caso contrario los convertiré en 'unknown'.

In [61]:
data.Sex.value_counts()

M      4982
F       624
M         2
N         2
lli       1
.         1
Name: Sex, dtype: int64

In [62]:
data['Sex'] = data['Sex'].replace({'M ': 'M', 'lli': 'M', 'N': 'M', '.': 'unknown'})

data['Sex'].fillna(value='unknown', inplace=True)

In [63]:
data.Sex.value_counts()

M          4987
F           624
unknown     558
Name: Sex, dtype: int64

Para la columna 'Age', aun siendo consciente de que yendo caso por caso podría rellenarla mejor, he decidido aplicar lo siguiente:

In [64]:
data.Age.value_counts()

17                154
18                150
19                141
20                140
15                137
                 ... 
7      &    31      1
 28                 1
20?                 1
 30                 1
2½                  1
Name: Age, Length: 156, dtype: int64

Primero reemplazo los espacios vacíos con '' (nada), de forma que las edades correctas que simplemente tengan algún espacio puedan unificarse correctamente.

In [65]:
data['Age'] = data['Age'].str.replace(' ', '')

Después reemplazo cualquier carácter no númerico por ''.

In [66]:
data['Age'] = data['Age'].str.replace('[^0-9]', '')

Ahora que tengo una lista de números, para todos aquellos que contengan más de dos dígitos decido aplicar una función y convertirlos en 'unknown'.

In [67]:
data['Age'] = data['Age'].astype(str)   # Convierto los valores de 'Age' a str para poder aplicar la función

def no_age_unknown(x):
    
    if len(x) > 2:
        
        return 'unknown'
    
    return x

data['Age'] = data['Age'].apply(no_age_unknown)

Finalmente reemplazo los espacios vacíos dejados por los antiguos valores que sólo contenían letras por 'unknown'.

In [68]:
data['Age'].replace("", "unknown", inplace=True)

In [69]:
data.Age.value_counts()

unknown    2776
17          154
18          151
20          150
19          142
           ... 
67            1
84            1
86            1
82            1
2             1
Name: Age, Length: 82, dtype: int64

En la columna 'Injury' me pasa un poco como en la columna 'Name', que considero que unificar la información es un trabajo línea por línea, y eliminarlo no tendría sentido porque perdería información. Siempre puedo después filtrar por palabras claves como 'laceration', 'right arm' o 'bitten', si quiero buscar un tipo de lesión en concreto. Lo que sí puedo hacer es unificar los FATAL o fatal, de forma que pueda comparar mejor la columna siguiente 'Fatal'. Además, convertiré los casos no informados (NaN) en 'unknown'.

In [70]:
data['Injury'] = np.where(data['Injury'].str.contains('FATAL'), 'FATAL', data['Injury'])

In [71]:
data['Injury'] = np.where(data['Injury'].str.contains('atal'), 'FATAL', data['Injury'])

In [72]:
data['Injury'].fillna(value='unknown', inplace=True)

Estudiando la siguiente columna 'Fatal', me doy cuenta de que puedo mejorar el ajuste de los valores de una columna usando los de la otra y viceversa. Aunque, para empezar, lo primero será ajustar los valores mal informados.

In [73]:
data.Fatal.value_counts()

N          4225
Y          1326
UNKNOWN      70
 N            7
M             1
2017          1
N             1
y             1
Name: Fatal, dtype: int64

In [74]:
data['Fatal'] = data['Fatal'].replace({' N': 'N', 'M': 'N', '2017': 'N', 'N ': 'N', 'y': 'Y'})

A continuación sustituyo por 'Y' los valores en los cuales el valor correspondiente de 'Injury' sea 'FATAL' y viceversa.

In [75]:
data['Fatal'] = np.where(data['Injury'] == 'FATAL', 'Y', data['Fatal'])

In [76]:
data.loc[data['Fatal'] == 'Y', 'Injury'] = 'FATAL'

In [77]:
data.Fatal.value_counts()

N          4230
Y          1410
UNKNOWN      51
Name: Fatal, dtype: int64

Tras analizar las filas restantes que a pesar de los cambios contienen 'UNKNOWN', me percato de que la mayoría coinciden con un 'No details' o similar en 'Injury'. Decido convertir esos valores de ambas columnas en 'unknown', además de rellenar los valores nulos que queden del mismo modo.

In [78]:
data['Injury'] = np.where(data['Fatal'] == 'UNKNOWN', 'unknown', data['Injury'])

In [79]:
data['Fatal'].fillna('unknown', inplace=True)

Turno para la columna 'Time'. Exploro sus valores.

In [80]:
data.Time.value_counts().tail(40)

12h34                                   1
8:04 pm                                 1
12h46                                   1
Late morning                            1
13h06                                   1
13h14                                   1
After Dusk                              1
11h57                                   1
Possibly same incident as 2000.08.21    1
12h35                                   1
17h42                                   1
10h28                                   1
18h25                                   1
13h345                                  1
                                        1
06h47                                   1
07h08                                   1
                                        1
11h115                                  1
"Just before 11h00"                     1
12h39                                   1
13h42                                   1
22h30                                   1
Just before sundown               

Primero creo una función 'clean_time' que me permita convertir las strings que dan información en hora númerica, y doy homogeneidad a los valores sustituyendo 'h' por ':'. 

In [81]:
def clean_time(x):
    
    x = x.replace('h', ':').replace('am', '').strip()
    
    if ':' not in x:
        
        if 'unchtime' in x:
            return '12:00'
        elif 'fternoon' in x:
            return '16:00'
        elif 'idnight' in x:
            return '23:59'
        elif 'orning' in x:
            return '09:00'
        elif 'usk' in x:
            return '19:00'
        elif 'vening' in x:
            return '17:00'
        elif 'ight' in x:
            return '21:00'
        elif 'idday' in x:
            return '12:00'
        else:
            return 'unknown'
    else:
        return x

In [82]:
data['Time'] = data['Time'].astype(str)   # Convierto los valores de 'Age' a str para poder aplicar la función

data['Time'] = data['Time'].apply(clean_time)

Tras ello, decido limpiar el resto de carácteres no númericos.

In [83]:
data['Time'] = data['Time'].str.replace('[^0-9:]', '')  

Entonces creo un diccionario con los valores restantes que siguen sin el formato adecuado y los sustituyo manualmente.

In [84]:
data['Time'] = data['Time'].replace({':': '', '10:4511:15': '11:00', '07:0008:00': '07:30', '18:1518:30': '18:22', '06:0008:': '07:00',
                                 '17:0017:40': '17:20', ':13:00': '13:00', '14:3015:30': '15:00', '09:0010:00': '09:30', '13:345': '13:34', '9:00': '09:00',
                                '05:0008:00': '06:30', '17:0018:00': '17:30', '10:3013:30': '12:00', '06:0007:00': '06:30', '11:01:': '11:01',
                                 ':03:10': '03:10', '11:0012:00': '11:30', '18:1521:30': '20:00', '10:0014:00': '12:00', '12:0014:00': '13:00', '08:0009:30': '08:45', '09:3015:30': '12:30', '12:4513:45': '13:15',
                                '03:4504:00': '03:52', '15:0015:45': '15:22', '09:3010:00': '09:45', '16:3018:00': '17:15', '8:04': '08:04', '10:0011:00': '10:30', '2:': '02:00',
                                '::': '', '14:0015:00': '14:30', '09:0009:30': '09:15', '06:0007:20': '06:40', ':12:00': '12:00', '11:0011:30': '11:15', '19:0020:00': '19:30',
                                    '11:115': '11:15'})

Ahora sólo me falta cambiar los vacíos por 'unknown' y darle un formato adecuado a la hora.

In [85]:
data['Time'].replace("", "unknown", inplace=True)

In [86]:
mask5 = data['Time'] != 'unknown'

data.loc[mask5, 'Time'] = pd.to_datetime(data.loc[mask5, 'Time'], format='%H:%M').dt.time

En la columna 'Species', hay muchísimos valores únicos, por lo que convertir el tipo en algo menos que object (por ej, categoría) parece complicado. Sí puedo convertir los NaN en 'unknown', por un lado, y por otro intentar unificar algo las especies de tiburón más repetidas, aun consciente de que algún error puede darse si dos palabras clave están en el mismo valor. Procedo.

In [87]:
data.Species.fillna('unknown', inplace=True)

In [88]:
data['Species'] = np.where(data['Species'].str.contains('hite'), 'White shark', data['Species'])

data['Species'] = np.where(data['Species'].str.contains('iger'), 'Tiger shark', data['Species'])

data['Species'] = np.where(data['Species'].str.contains('ull'), 'Bull shark', data['Species'])

data['Species'] = np.where(data['Species'].str.contains('lue'), 'Blue shark', data['Species'])

data['Species'] = np.where(data['Species'].str.contains('urse'), 'Nurse shark', data['Species'])

data['Species'] = np.where(data['Species'].str.contains('ako'), 'Mako shark', data['Species'])

data['Species'] = np.where(data['Species'].str.contains('hammer'), 'Hammerhead shark', data['Species'])

La columna 'Source' tiene demasiados valores únicos como para considerar cambiarla en este momento, así que procedo a rellenar los valores nulos.

In [89]:
data['Source'].fillna('unknown', inplace=True)

La columna 'pdf' parece contener el nombre del archivo adjunto en los links de 'href' y 'href formula', y además no tiene ningún valor nulo, y por tanto la voy a dejar como está

Sin enmbargo, la columna 'href_formula' sí tiene un valor nulo. Lo encuentro con el siguiente código:

In [90]:
data.loc[data['href_formula'].isnull()]

,Day,Month,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Time,Species,Source,pdf,href_formula,href,Case_Number_1,Case_Number_2,original_order,Unnamed_22,Unnamed_23
3214,19,1,1975,Unprovoked,AUSTRALIA,South Australia,Coffin Bay,Surfing,David Barrowman,M,17,FATAL,Y,unknown,unknown,"J. West; Adelaide Advertiser, 1/20/1975; P. Kemp, GSAF",1975.01.19-Barrowman.pdf,NaN,http://sharkattackfile.net/spreadsheets/pdf_directory/1975.01.19-Barrowman.pdf,1975.01.19,1975.01.19,3059,NaN,NaN


El índice es el 3214, y voy a rellenar ese nulo con los datos de la columna 'href', pues son iguales a los de 'href_formula' salvo por esta omisión.

In [91]:
data.href_formula.iloc[3214] = data.href.iloc[3214]

In [92]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6169 entries, 0 to 6168
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Day             6169 non-null   int32 
 1   Month           6169 non-null   int32 
 2   Year            6169 non-null   int32 
 3   Type            6169 non-null   object
 4   Country         6169 non-null   object
 5   Area            6169 non-null   object
 6   Location        6169 non-null   object
 7   Activity        6169 non-null   object
 8   Name            6169 non-null   object
 9   Sex             6169 non-null   object
 10  Age             6169 non-null   object
 11  Injury          6169 non-null   object
 12  Fatal           6169 non-null   object
 13  Time            6169 non-null   object
 14  Species         6169 non-null   object
 15  Source          6169 non-null   object
 16  pdf             6169 non-null   object
 17  href_formula    6169 non-null   object
 18  href    

Las columnas 'Case_Number_1' y 'Case_Number_2' contienen una vez más las fechas que ya hemos adecuado en las primeras columnas. A la columna 'Case_Number_1' la voy a convertir en simplemente 'Case_Number', donde voy a dejar guardado el número de caso en orden descendente, ya que no coincidirá con 'original_order', la cual he decidido mantener como está para saber el orden original del caso. 

A la columna 'Case_Number_2' la voy a convertir en una columna de fecha llamada 'Date' con los valores de 'Day', 'Month' y 'Year' sumados.

Finalmente y para no volver a usar el .rename, aprovecho y renombro las dos columnas finales para avisar de lo que son y prevenir su uso.

In [93]:
data.rename(columns={'Case_Number_1': 'Case_Number', 'Case_Number_2': 'Date',
                     'Unnamed_22': 'no_data_1', 'Unnamed_23': 'no_data_2'}, inplace=True)

In [94]:
data['Case_Number'] = [len(data) - i for i in range(len(data))]

In [95]:
data['Date'] = data['Year'].astype(str) + '-' + data['Month'].astype(str) + '-' + data['Day'].astype(str)

Aunque ahora pretendía cambiar el tipo de la columna a tipo fecha, al hacerlo no me permite convertir las últimas filas, y según he investigado podría ser porque son demasiado lejanas en el tiempo. Puedo forzar a hacerlo al código usando un errors=coerce, pero entonces esos valores son NaT. Así que, de momento y hasta que haga el análisis de tipos para optimizar DataFrame, no cambio el formato a fecha.

Como especificado anteriormente, la columna 'original_order' la dejo como está, así que sólo faltaría ajustar los valores de las últimas dos columnas. He decidido que sean 0 para que pesen menos y, al comprobar el número de 'unknown's por fila, no computen.

In [96]:
data.no_data_1 = 0

In [97]:
data.no_data_2 = 0

Para terminar quiero comprobar si alguna fila tiene más del 50% de 'unknown's sobre 22 columnas, ya que las últimas dos no conviene tenerlas en cuenta.

In [98]:
((data == 'unknown').sum(axis=1) > 11).unique()

array([False])

Ninguna fila tiene más de la mitad de 'unknown', por lo que no elimino ninguna y el DataFrame se queda como está. Ahora solo falta intentar ajustar los tipos de dato.

In [99]:
for c in data.select_dtypes('object'):
    
    data[c]=data[c].astype('category')

In [100]:
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6169 entries, 0 to 6168
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Day             6169 non-null   int32   
 1   Month           6169 non-null   int32   
 2   Year            6169 non-null   int32   
 3   Type            6169 non-null   category
 4   Country         6169 non-null   category
 5   Area            6169 non-null   category
 6   Location        6169 non-null   category
 7   Activity        6169 non-null   category
 8   Name            6169 non-null   category
 9   Sex             6169 non-null   category
 10  Age             6169 non-null   category
 11  Injury          6169 non-null   category
 12  Fatal           6169 non-null   category
 13  Time            6169 non-null   category
 14  Species         6169 non-null   category
 15  Source          6169 non-null   category
 16  pdf             6169 non-null   category
 17  href_formula  

In [101]:
for c in data.select_dtypes('integer'):
    
    data[c]=pd.to_numeric(data[c], downcast='integer')
    
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6169 entries, 0 to 6168
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Day             6169 non-null   int8    
 1   Month           6169 non-null   int8    
 2   Year            6169 non-null   int16   
 3   Type            6169 non-null   category
 4   Country         6169 non-null   category
 5   Area            6169 non-null   category
 6   Location        6169 non-null   category
 7   Activity        6169 non-null   category
 8   Name            6169 non-null   category
 9   Sex             6169 non-null   category
 10  Age             6169 non-null   category
 11  Injury          6169 non-null   category
 12  Fatal           6169 non-null   category
 13  Time            6169 non-null   category
 14  Species         6169 non-null   category
 15  Source          6169 non-null   category
 16  pdf             6169 non-null   category
 17  href_formula  

Para finalizar, imprimo el DataFrame final y lo exporto a carpeta.

# DataFrame Final

In [103]:
data.shape, data_ori_shape

((6169, 24), (25723, 24))

In [105]:
data.head()

,Day,Month,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Time,Species,Source,pdf,href_formula,href,Case_Number,Date,original_order,no_data_1,no_data_2
0,25,6,2018,Boating,USA,California,"Oceanside, San Diego County",unknown,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and paddle damaged",N,18:00:00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.06.25-Wolfe.pdf,6169,2018-6-25,6303,0,0
1,23,2,2018,Unprovoked,AUSTRALIA,New South Wales,"Little Congwong Beach, La Perouse",Swimming,Anna Shurapey,F,55,Laceratons to right leg & foot,N,19:00:00,White shark,"B. Myatt, GSAF",2018.02.23-Shurapey.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.02.23-Shurapey.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.02.23-Shurapey.pdf,6168,2018-2-23,6263,0,0
2,15,4,2018,Unprovoked,AUSTRALIA,Western Australia,"Cobblestones, Margaret River Area",Surfing,Alejandro Travaglini,M,37,Lacerations to legs,N,08:00:00,unknown,"B.Myatt, GSAF",2018.04.15.a-Travaglini.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.04.15.a-Travaglini.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.04.15.a-Travaglini.pdf,6167,2018-4-15,6274,0,0
3,14,4,2018,Unprovoked,BAHAMAS,New Providence,Nirvana Beach,Surfing,Bruce Rowan,M,unknown,No Injury. Shark swam away with the surf board,N,09:30:00,Tiger shark,"Tribune242,",2018.04.14-Rowan.pff,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.04.14-Rowan.pff,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.04.14-Rowan.pff,6166,2018-4-14,6273,0,0
4,10,1,2018,Invalid,BRAZIL,Alagoas,"Praia de Sauaçuhy, Maceió",Fishing,Josias Paz,M,56,Injury to ankle from marine animal trapped in weir PROVOKED INCIDENT.,N,unknown,Shark involvement not confirmed,"K. McMurray, TrackingSharks.com",2018.04.10.R-Paz.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.04.10.R-Paz.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2018.04.10.R-Paz.pdf,6165,2018-1-10,6272,0,0


In [102]:
data.to_csv('clean_sharks.csv', index=False)